**Decision Tree -- Theory**

**Random Forest -- Theory**

As the name implies, a random forest is built from an **ensemble** of decision trees. While this method improves the accuracy of the classifier/regressor, the results are clearly less interpretable than they are in a single decision tree.

So how do we create more than one tree? The naive approach would be to repeat the above algorithm for decision trees multiple times, but this would end up creating highly-correlated trees. Instead, we can **bag** the data: for each tree, choose $n$ samples from the training set (*with* replacement).

Now, if any features are strong predictors across the data set, they will likely be chosen earlier in the learning process for multiple trees, which may cause the trees to become correlated. To remedy this, at each step (node) of the learning process for the decision tree, consider only $d$ distinct features, and select the best feature to split the node.

To use this model for evaluation, we can take an aggregate result of the decision trees. For a classification prediction, this involves a simple majority vote of the trees' predicted values. For a regression problem, the model takes the average of all individual regression trees.

**Implementation**

Now, we will work on implementing the Decision Tree and Random Forest architectures. The first step is importing the packages and methods we will need.

In [2]:
# Dependencies
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import accuracy_score, recall_score, f1_score, mean_squared_error, r2_score

The next step is making the data. We will use two datasets for this project. ___ To make the data easier to work with, we will convert the csv files into pandas dataframes. We will also seperate the target variables (i.e. the y values) into seperate objects. 

In [4]:
# Process CSV files into dataframes
cancer_df = pd.read_csv('data/breast_cancer.csv')
cancer_df.pop('ID')
solar_df = pd.read_csv('data/solar.csv')

# Make target objects
cancer_y = cancer_df.pop('diagnosis')
solar_y = solar_df.pop('SOLARRADIATION_0003')
solar_df = solar_df.iloc[:,1:]

# Print results
cancer_df.head()
solar_df.head()

,DSWRF_SFC_0000,DSWRF_SFC_0001,DSWRF_SFC_0002,DSWRF_SFC_0003,DSWRF_SFC_0004,DSWRF_SFC_0005,DSWRF_SFC_0006,DSWRF_SFC_0007,DSWRF_SFC_0008,DSWRF_SFC_0009,...,VIS_SFC_0019,VIS_SFC_0020,VIS_SFC_0021,VIS_SFC_0022,VIS_SFC_0023,VIS_SFC_0024,time_of_day_cos,time_of_day_sin,time_of_year_cos,time_of_year_sin
0,0.0,0.000,0.000,0.000,0.00,0.0,0.00,1.75,7.750,18.25,...,426.895569,227.899689,228.550201,428.331696,227.286820,224.271637,0,1,0.977848,0.209315
1,106.5,63.875,90.750,17.875,10.00,0.0,0.00,0.00,0.000,0.00,...,7824.965820,4825.635742,2627.959229,2628.863281,4828.833008,9027.083984,0,-1,0.976011,0.217723
2,0.0,0.000,0.000,0.000,0.00,0.0,0.00,0.00,0.000,0.00,...,24226.927730,24223.308590,14824.718750,20222.730470,16624.769530,19425.916020,1,0,0.975065,0.221922
3,0.0,0.000,0.000,0.000,0.00,0.0,0.00,6.50,60.125,50.25,...,5625.299316,8026.288086,20024.955080,6023.587402,24223.126950,24222.828130,0,1,0.974100,0.226116
4,0.0,8.625,105.625,55.375,140.75,100.0,77.25,56.25,80.125,44.50,...,23823.607420,24223.201170,10823.557620,17022.439450,5824.313965,24222.529300,-1,0,0.973118,0.230306


Another imporant part of data processing is normalizing the data. This usually means putting each data value in the dataframe somewhere between one and zero. Often, this is done by subtracting the observation from the maximum observation and then dividing by the range ($scaled=\frac{max - unscaled}{max - min}$). We will use a built in sklearn function to do this for us.

In [ ]:
def scale(df):
    x = df.values
    scaler = MinMaxScaler()
    x_scaled = scaler.fit_transform(x)
    return pd.DataFrame(x_scaled)

Now we will define the method for making a model. You will see the method `makeModel` takes five parameters: `num_features, model, df, y, clf`. `num_features` is the number of input features to consider, as we will only use the n features containing the highest variance. The model `model` is specifies the predefined sklearn model we will use. `df` is the dataframe we will use, `y` is the corresponding target variable, and `clf` is True if we are performing a classfiication task and False if we are performing a regression task.  

First, we preprocess the data by normalizing, choosing for features with the highest variance, and splitting into training and testing (we use an 80/20 split). Next, we train the model (using skelarn's `.fit` function) and predict results for the testing data. Lastly, we will calculate metrics to measure the success of the predictions on the testing data. If it's a classification task, we will calculate accuracy, recall, and f1 score. If it is a regression task, we will caclulate mean squared error and the $r^2$ score. If you are not sure how any of these scores are calculated, take a minute to research them. Which of these metrics do you think are most important? Also, notice how many tasks we can simply use the built-in sklearn method for. 

In [ ]:
def makeModel(num_features, model, df, y, clf):
    # Restrict to only features with highest variance + other preprocessing
    cols = df.var().sort_values()[(-num_features - 1):].keys()
    df = df[df.columns.intersection(cols)]
    df = scale(df)
    training_x, testing_x, training_y, testing_y = train_test_split(df, y, test_size=0.2)
    
    # Fit model and predict results
    model.fit(training_x, training_y)
    pred = model.predict(testing_x)
    print(str(model) + " with " + str(num_features) + " features: ")
    
    if clf:       
        # Test for accuracy, recall, and f1 score
        accuracy = accuracy_score(testing_y, pred)
        recall = recall_score(testing_y, pred, pos_label = 'M') #Double Check? 
        f1 = f1_score(testing_y, pred, pos_label = 'M')
        print ("\tAccuracy of " + str(accuracy))
        print ("\tRecall of " + str(recall))
        print ("\tF1 of " + str(f1))
    else:
        # Test for mean squared error and r2 score.
        mse = mean_squared_error(testing_y, pred)
        r2 = r2_score(testing_y, pred)
        print("\tMean Squared Error of " + str(mse))
        print("\tR2 Score of " + str(r2))

In [ ]:
classifiers = [DecisionTreeClassifier(), LogisticRegression(), RandomForestClassifier()]
regressors = [DecisionTreeRegressor(), LinearRegression(), RandomForestRegressor()]
for ftrs in range(5, 20, 5):
    for clf in classifiers:
        makeModel(ftrs, clf, cancer_df, cancer_y, True)
    for reg in regressors:
        makeModel(ftrs, reg, solar_df, solar_y, False)

**Sources**

CSCI 6380 with Dr. Fred Maier

MIT Open Courseware